# Naiwny Klasyfikator Bayesowski - klasyfikacja spamu

In [71]:
# !wget https://github.com/betacord/ML/archive/refs/heads/master.zip

--2023-04-23 15:03:48--  https://github.com/betacord/ML/archive/refs/heads/master.zip
Translacja github.com (github.com)... 140.82.121.3
Łączenie się z github.com (github.com)|140.82.121.3|:443... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 302 Found
Lokalizacja: https://codeload.github.com/betacord/ML/zip/refs/heads/master [podążanie]
--2023-04-23 15:03:49--  https://codeload.github.com/betacord/ML/zip/refs/heads/master
Translacja codeload.github.com (codeload.github.com)... 140.82.121.10
Łączenie się z codeload.github.com (codeload.github.com)|140.82.121.10|:443... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 200 OK
Długość: nieznana [application/zip]
Zapis do: ‘master.zip.1’

master.zip.1            [         <=>        ]  13,63M  8,21MB/s    w 1,7s     

2023-04-23 15:03:51 (8,21 MB/s) - zapisano ‘master.zip.1’ [14293941]



In [72]:
# !unzip master.zip

Archive:  master.zip
00a33d80fb0de76fb3e87c41db948c3881f9acbb
replace ML-master/.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [105]:
import email
import io
import re
import os
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup

## klasa EmailObject

parsowanie i reprezentacja wiadomości email

In [106]:
class EmailObject:
  def __init__(self, file, category=None):
    self.mail = email.message_from_file(file)
    self.category = category

  def subject(self):
    return self.mail.get('Subject')

  def body(self):
    content_type = self.mail.get_content_type()
    body = self.mail.get_payload(decode=False)

    # import pdb; pdb.set_trace()
    if content_type == 'text/html':
      return BeautifulSoup(body, 'html.parser').text
    elif content_type == 'text/plain':
      return body
    else:
      return ''

In [107]:
email_obj_0 = EmailObject(io.open('master/ML-master/lab2/data/TRAINING/TRAIN_00001.eml', 'r', encoding='latin-1'))

In [108]:
email_obj_0.subject()

'link to my webcam you wanted '

In [109]:
email_obj_0.body()

'Wanna see sexually curious teens playing with each other?\n\nhttp://www.site-personals.com <-- click here=)\n\nme and my horny girlfriends are waiting for you... we are probably eating each other out on webcam in our dormitory as ur reading this! (inbetween classes of course *wink*)\n\nsee you soon baby,\n-Veronica\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nmcmfkhcpedgetqj\n\n'

## klasa Tokenizer

tokenizacja tekstu

In [110]:
class Tokenizer:
  NULL = u'\u0000'

  @staticmethod
  def tokenize(txt):
    return re.findall('\w+', txt.lower())

  @staticmethod
  def unique_tokenizer(txt):
    tokens = Tokenizer.tokenize(txt)
    return set(tokens)

In [111]:
tokens = Tokenizer.unique_tokenizer('Ala ma kota, a kot ma ale')

In [112]:
tokens

{'a', 'ala', 'ale', 'kot', 'kota', 'ma'}

## klasa SpamTrainer

- uczenie modelu
- klasyfikacja

In [113]:
class SpamTrainer:
  def __init__(self, training_files):
    self.categories = set()

    for category, file in training_files:
      self.categories.add(category)

    self.totals = defaultdict(float)
    self.training = {c: defaultdict(float) for c in self.categories}
    self.to_train = training_files

  def total_for(self, category):
    return self.totals[category]

  def train(self):
    for category, file in self.to_train:
      filepath = f'master/ML-master/lab2/data/TRAINING/{os.path.basename(file)}'

      with io.open(filepath, 'r', encoding='latin-1') as f:
        mail = EmailObject(f)

      self.categories.add(category)

      for token in Tokenizer.unique_tokenizer(mail.body()):
        self.training[category][token] += 1
        self.totals['_all'] += 1
        self.totals[category] += 1

    self.to_train = {}

  def score(self, mail):
    self.train()
    cat_totals = self.totals

    aggregates = {c: cat_totals[c] / cat_totals['_all'] for c in self.categories}

    for token in Tokenizer.unique_tokenizer(mail.body()):
      for cat in self.categories:
        value = self.training[cat][token]
        r = (value + 1) / (cat_totals[cat] + 1)
        aggregates[cat] *= r

    return aggregates

  def normalized_score(self, mail):
    score = self.score(mail)
    scoresum = sum(score.values())

    normalized = {cat: (agg / scoresum) for cat, agg in score.items()}

    return normalized

  def preference(self):
    return sorted(self.categories, key=lambda cat: self.total_for(cat))

  class Classification:
    def __init__(self, guess, score):
      self.guess = guess
      self.score = score

    def __eq__(self, other):
      return self.guess == other.guess and self.score == other.score

  def classify(self, mail):
    score = self.score(mail)

    max_score = 0.0
    preference = self.preference()
    max_key = preference[-1]

    for k, v in score.items():
      if v > max_score:
        max_key = k
        max_score = v
      elif v == max_score and preference.index(k) > preference.index(max_key):
        max_key = k
        max_value = v
    
    return self.Classification(max_key, max_score)

In [114]:
training_data = [
    ('ham', 'master/ML-master/lab2/data/TRAINING/TRAIN_00002.eml'),
    ('spam', 'master/ML-master/lab2/data/TRAINING/TRAIN_00000.eml'),
    ('ham', 'master/ML-master/lab2/data/TRAINING/TRAIN_00006.eml'),
    ('spam', 'master/ML-master/lab2/data/TRAINING/TRAIN_00003.eml'),
]

spam_trainer = SpamTrainer(training_data)

In [115]:
email_obj0_score = spam_trainer.classify(email_obj_0)

## walidacja modelu

In [116]:
def label_to_training_data(fold_file):
  training_data = []

  with open(fold_file, 'r') as f:
    for line in f:
      target, filepath = line.rstrip().split(' ')
      training_data.append((target, filepath))

  return SpamTrainer(training_data)

In [117]:
def parse_emails(keyfile):
  emails = []

  with open(keyfile, 'r') as f:
    for line in f:
      label, file = line.rstrip().split(' ')

      filepath = f'master/ML-master/lab2/data/TRAINING/{os.path.basename(file)}'
      
      with io.open(filepath, 'r', encoding='latin-1') as labelfile:
        emails.append(EmailObject(labelfile, category=label))

  
  return emails

In [118]:
def validate(trainer, set_of_emails):
  correct = 0
  false_positives = 0.
  false_negatives = 0.
  confidence = 0.

  for mail in set_of_emails:
    classification = trainer.classify(mail)
    confidence += classification.score

    if classification.guess == 'spam' and mail.category == 'ham':
      false_positives += 1
    elif classification.guess == 'ham' and mail.category == 'spam':
      false_negatives += 1
    else:
      correct += 1

  total = false_positives + false_negatives + correct

  false_positive_rate = false_positives / total
  false_negative_rate = false_negatives / total
  err = (false_positives + false_negatives) / total

  return false_positive_rate, false_negative_rate, err

In [119]:
trainer = label_to_training_data('master/ML-master/lab2/data/fold2.label')
emails = parse_emails('master/ML-master/lab2/data/fold1.label')

In [120]:
fpr, fnr, err = validate(trainer, emails)

In [121]:
f'FPR: {fpr}, FNR: {fnr}, error: {err}'

'FPR: 0.0018484288354898336, FNR: 0.20841035120147874, error: 0.21025878003696857'

In [122]:
spam_trainer.classify(email_obj_0)

In [123]:
trainer

In [124]:
# zadanie samodzielne

In [125]:
with open("master/ML-master/lab2/data/fold_suma.label" ,'a') as f_sum:
  with open('master/ML-master/lab2/data/fold2.label', 'r') as f2:
    with open("master/ML-master/lab2/data/fold1.label","r") as f1:
      #Read first file
      f2_lines = f2.readlines()
      print(f2_lines)  
      f_sum.writelines(f2_lines)  
      #Read second file
      f2_lines = f2.readlines()
      f_sum.writelines(f2_lines)  
      #Combine content of both lists
      #combine = list(zip(ylines,xlines))
      #Write to third file
      

['ham ./data/TRAINING/TRAIN_00005.eml\n', 'spam ./data/TRAINING/TRAIN_00001.eml\n', 'ham ./data/TRAINING/TRAIN_00007.eml\n', 'spam ./data/TRAINING/TRAIN_00004.eml\n', 'ham ./data/TRAINING/TRAIN_00009.eml\n', 'spam ./data/TRAINING/TRAIN_00013.eml\n', 'ham ./data/TRAINING/TRAIN_00012.eml\n', 'spam ./data/TRAINING/TRAIN_00015.eml\n', 'ham ./data/TRAINING/TRAIN_00017.eml\n', 'spam ./data/TRAINING/TRAIN_00022.eml\n', 'ham ./data/TRAINING/TRAIN_00019.eml\n', 'spam ./data/TRAINING/TRAIN_00027.eml\n', 'ham ./data/TRAINING/TRAIN_00023.eml\n', 'spam ./data/TRAINING/TRAIN_00033.eml\n', 'ham ./data/TRAINING/TRAIN_00026.eml\n', 'spam ./data/TRAINING/TRAIN_00040.eml\n', 'ham ./data/TRAINING/TRAIN_00029.eml\n', 'spam ./data/TRAINING/TRAIN_00042.eml\n', 'ham ./data/TRAINING/TRAIN_00032.eml\n', 'spam ./data/TRAINING/TRAIN_00050.eml\n', 'ham ./data/TRAINING/TRAIN_00035.eml\n', 'spam ./data/TRAINING/TRAIN_00059.eml\n', 'ham ./data/TRAINING/TRAIN_00037.eml\n', 'spam ./data/TRAINING/TRAIN_00065.eml\n', 'ha

In [126]:
# Podziały danych
import pandas as pd
from random import sample
df = pd.read_csv('master/ML-master/lab2/data/fold_suma.label', header=None)

In [127]:
test_rate =.45

In [128]:
df

,0
0,ham ./data/TRAINING/TRAIN_00005.eml
1,spam ./data/TRAINING/TRAIN_00001.eml
2,ham ./data/TRAINING/TRAIN_00007.eml
3,spam ./data/TRAINING/TRAIN_00004.eml
4,ham ./data/TRAINING/TRAIN_00009.eml
...,...
6484,ham ./data/TRAINING/TRAIN_04309.eml
6485,ham ./data/TRAINING/TRAIN_04311.eml
6486,ham ./data/TRAINING/TRAIN_04316.eml
6487,ham ./data/TRAINING/TRAIN_04320.eml


In [129]:
# test_count= round(len(df)*test_rate)

In [130]:
# test_rows = sample(list(df.index.values),test_count)

In [131]:
# train_rows = list(set(df.index.values) - set(test_rows))

In [132]:
# train_df = df.loc[train_rows]
# test_df = df.loc[test_rows]

In [133]:
def split_df(df, test_rate):
    test_count=round(len(df)*test_rate)
    test_rows = sample(list(df.index.values),test_count)
    train_rows = list(set(df.index.values) - set(test_rows))
    train_df = df.loc[train_rows]
    test_df = df.loc[test_rows]
    return train_df, test_df

In [134]:
train_df, test_df  = split_df(df, .45)


In [135]:
# #trainer = label_to_training_data('master/ML-master/lab2/data/fold2.label')
# emails = parse_emails('master/ML-master/lab2/data/fold1.label')

In [139]:
# trainer csv
train_df.to_csv('master/ML-master/lab2/data/trainer.csv', sep= ' ', index=None, header=None)
label_to_training_data('master/ML-master/lab2/data/trainer.csv')


In [142]:
# test csv
test_df.to_csv('master/ML-master/lab2/data/test.csv', sep= ' ', index=None, header=None)


In [148]:
trainer = label_to_training_data('master/ML-master/lab2/data/trainer.csv')
emails = parse_emails('master/ML-master/lab2/data/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'master/ML-master/lab2/data/TRAIN_00211.eml"'

In [ ]:
fpr, fnr, err = validate(trainer, emails)